<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/autogen/AutoGen_v0_7_4_Quickstart_OpenAI_ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGen v0.7.4 Quickstart (OpenAI 버전 · Colab 실습)

이 노트북은 Microsoft AutoGen v0.7.4의 **AgentChat Quickstart**를 OpenAI 모델 기반으로 한글 실습 예제로 재작성한 것입니다.

## 목표
- OpenAI GPT 모델(`gpt-4o`, `gpt-4o-mini` 등)을 사용하는 **AssistantAgent** 구성
- 파이썬 함수를 도구(Tool)로 등록하여 에이전트가 직접 호출
- `.run_stream()`을 활용한 실시간 스트리밍 응답 확인
- Colab에서 한글 환경으로 실습

In [1]:
# 0) 필수 라이브러리 설치
!pip install -q "autogen-agentchat" "autogen-ext[openai]" nest_asyncio
import nest_asyncio
nest_asyncio.apply()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 33.6 MB/s eta 0:00:00


## 1. OpenAI API 키 설정
- OpenAI 모델을 사용하기 위해 **OPENAI_API_KEY** 환경 변수를 설정해야 합니다.
- Colab에서는 아래 셀에서 직접 입력하거나, **런타임 > 환경 변수** 메뉴를 사용하세요.

In [2]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

## 2. 모델 클라이언트 생성
- OpenAI의 ChatCompletionClient를 사용하여 모델 클라이언트를 생성합니다.
- 기본값은 `gpt-4o-mini`이지만 필요에 따라 변경 가능합니다.

In [3]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"  # gpt-4o, gpt-4.1, gpt-4o-mini 등 교체 가능
)
print("✅ OpenAI 모델 클라이언트 초기화 완료")

✅ OpenAI 모델 클라이언트 초기화 완료


## 3. 도구(Tool) 정의
- 에이전트가 호출할 수 있는 함수를 등록합니다.
- AutoGen은 **파라미터 타입힌트**를 활용해 자동으로 LLM 함수콜을 매핑합니다.

In [4]:
async def get_weather(city: str) -> str:
    """도시명을 입력받아 날씨 정보를 반환합니다."""
    return f"현재 {city}의 날씨는 맑고 25도입니다."

print(await get_weather("서울"))

현재 서울의 날씨는 맑고 25도입니다.


## 4. AssistantAgent 생성
- OpenAI 모델 클라이언트와 툴을 등록하여 에이전트를 구성합니다.
- `reflect_on_tool_use=True` 설정 시, 도구 사용 후 자체 점검 단계가 추가됩니다.
- `model_client_stream=True` 설정으로 실시간 스트리밍 응답을 지원합니다.

In [5]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,
)
print("✅ AssistantAgent 초기화 완료")

✅ AssistantAgent 초기화 완료


## 5. 스트리밍 실행
- `.run_stream()` 메서드를 사용해 에이전트의 답변을 실시간으로 확인할 수 있습니다.
- `Console` UI를 사용하면 함수 호출 → 실행 결과 → 최종 답변 과정을 쉽게 추적할 수 있습니다.

In [6]:
import asyncio
from autogen_agentchat.ui import Console

async def main():
    await Console(agent.run_stream(task="뉴욕의 현재 날씨를 알려줘"))
    await model_client.close()

await main()

---------- TextMessage (user) ----------
뉴욕의 현재 날씨를 알려줘
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_xwA0k8fZkYHtWg78F41ZYrxd', arguments='{"city":"New York"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='현재 New York의 날씨는 맑고 25도입니다.', name='get_weather', call_id='call_xwA0k8fZkYHtWg78F41ZYrxd', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
현재 뉴욕의 날씨는 맑고 기온은 25도입니다.


## 6. 실습 과제
1) **새로운 도구 추가**: `get_weather` 외에 다른 함수를 만들어 추가해보세요. (예: 환율 조회, 계산기)
2) **프롬프트 가드레일 적용**: 시스템 메시지를 수정하여 에이전트의 답변 방식을 제어해보세요.
3) **다중 도구 실습**: 여러 함수를 등록하고 LLM이 자동으로 선택하도록 설정해보세요.
4) **스트리밍 분석**: `Console` 출력 로그를 통해 함수 호출 및 결과 과정을 분석해보세요.

## 7. 참고 링크
- [AutoGen AgentChat Quickstart 공식 문서](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/quickstart.html)
- [AutoGen GitHub 저장소](https://github.com/microsoft/autogen)
- [OpenAI API 키 발급](https://platform.openai.com/account/api-keys)
